In [1]:
import warnings
warnings.filterwarnings("ignore")

###  info_wt.csv : 웹툰의 정보 (타이틀, 작가, 홧수 등)
### comments.csv : 댓글의 정보 (유저id, 댓글, 댓글이 쓰여진 웹툰 등)

In [2]:
info_wt = pd.read_csv('info_wt.csv')
comments = pd.read_csv('comments.csv')

In [3]:
info_wt[:5]

,wt_num,title,score,artist,genre,chapter,reader
0,0,신의 탑,9.93,SIU,fantasy,380,982
1,1,뷰티풀 군바리,9.82,설이 / 윤성원,drama,161,2332
2,2,윈드브레이커,9.96,조용석,drama,212,550
3,3,소녀의 세계,9.98,모랑지,daily,141,355
4,5,평범한 8반,9.95,영파카,comic,213,485


In [4]:
comments[-5:]

,chapter,user_id,comment,like,dislike,wt_num
735496,16,정예****(yeji****),나라야...포기하면 협박 하겠어,0,0,191
735497,16,정예****(yeji****),나라야!!!니가 성윤이를 포기하면 내가 가지게 될것이야!!!!,0,1,191
735498,16,정예****(yeji****),나라하고 예진하고 성윤이를 두고 삼각 관계라면 내가 성윤이를 가지겠어!!!,0,3,191
735499,16,민트****(skie****),태호가 왜그래?하고 얼굴들이밀때 나도 깜놀 ㅋㅋ\n전에 성윤이가 최날이랑 버스 나란...,2,0,191
735500,16,이경****(jpmj****),옹,1,0,191


In [5]:
print('user의 수 : ', len(set(comments['user_id'])))
print('웹툰의 수  : ', len(set(comments['wt_num'])))
print('댓글의 수  : ', len(comments))

user의 수 :  39171
웹툰의 수  :  139
댓글의 수  :  735501


In [6]:
comments.describe()

,chapter,like,dislike,wt_num
count,735501.000000,735501.000000,735501.000000,735501.000000
mean,157.972385,58.724339,3.004727,87.920857
std,208.101928,1014.708363,39.715578,54.049251
min,1.000000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,47.000000
50%,121.000000,1.000000,0.000000,86.000000
75%,209.000000,3.000000,2.000000,130.000000
max,1215.000000,111989.000000,15576.000000,191.000000


In [7]:
import statsmodels.api as sm

In [8]:
y = info_wt[["reader"]]

In [9]:
model = sm.OLS.from_formula("reader ~ C(artist)", info_wt)
result = model.fit()
# print(result.summary())

In [10]:
comments_1 = comments[['user_id','wt_num']]
comments_1['count'] = 1
comments_1 = comments_1.groupby(['user_id','wt_num']).sum()

In [11]:
user_id = list(set(comments['user_id']))
wt_num = list(set(comments['wt_num']))

### 웹툰마다 댓글을 몇개 썼는지 보는 테이블

In [12]:
user_df = pd.DataFrame(data = 0,index = user_id, columns= wt_num)

In [13]:
for id in user_id :
    numwt = list(comments_1.loc[id].index)
    for num in numwt :
        user_df.loc[id][num] = int(comments_1.loc[id].loc[num])

In [14]:
display(user_df.head())
len(user_df)

,0,1,2,3,5,6,7,8,9,10,...,180,182,183,184,185,186,187,188,190,191
인간****(seoy****),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
이영****(soap****),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ye****(woai****),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
치느****(higu****),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
하령****(con9****),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


39171

### 댓글을 썼는지 안썼는지 만 판단하는 테이블

In [15]:
user_wt_matrix = user_df.copy()

In [16]:
user_wt_matrix1 = user_wt_matrix.copy()
for i in range(len(user_wt_matrix1)):
    try:
        for j in range(len(user_wt_matrix1[i])):
            if user_wt_matrix1[i][j] != 0:
                user_wt_matrix1[i][j] = 1
    except:
        pass

In [17]:
for i in range(20):
    n = len(user_wt_matrix1[user_wt_matrix1.sum(axis=1)>i])
    print(i+1, "개 이상의 웹툰에 댓글을 단 사람은",n, "명")

1 개 이상의 웹툰에 댓글을 단 사람은 39171 명
2 개 이상의 웹툰에 댓글을 단 사람은 12135 명
3 개 이상의 웹툰에 댓글을 단 사람은 5774 명
4 개 이상의 웹툰에 댓글을 단 사람은 3290 명
5 개 이상의 웹툰에 댓글을 단 사람은 2150 명
6 개 이상의 웹툰에 댓글을 단 사람은 1535 명
7 개 이상의 웹툰에 댓글을 단 사람은 1121 명
8 개 이상의 웹툰에 댓글을 단 사람은 873 명
9 개 이상의 웹툰에 댓글을 단 사람은 683 명
10 개 이상의 웹툰에 댓글을 단 사람은 535 명
11 개 이상의 웹툰에 댓글을 단 사람은 437 명
12 개 이상의 웹툰에 댓글을 단 사람은 345 명
13 개 이상의 웹툰에 댓글을 단 사람은 285 명
14 개 이상의 웹툰에 댓글을 단 사람은 236 명
15 개 이상의 웹툰에 댓글을 단 사람은 210 명
16 개 이상의 웹툰에 댓글을 단 사람은 184 명
17 개 이상의 웹툰에 댓글을 단 사람은 163 명
18 개 이상의 웹툰에 댓글을 단 사람은 141 명
19 개 이상의 웹툰에 댓글을 단 사람은 122 명
20 개 이상의 웹툰에 댓글을 단 사람은 104 명


In [18]:
for i in range(29,130,10):
    n = len(user_wt_matrix1[user_wt_matrix1.sum(axis=1)>i])
    print(i+1, "개 이상의 웹툰에 댓글을 단 사람은",n, "명")

30 개 이상의 웹툰에 댓글을 단 사람은 44 명
40 개 이상의 웹툰에 댓글을 단 사람은 20 명
50 개 이상의 웹툰에 댓글을 단 사람은 6 명
60 개 이상의 웹툰에 댓글을 단 사람은 4 명
70 개 이상의 웹툰에 댓글을 단 사람은 3 명
80 개 이상의 웹툰에 댓글을 단 사람은 1 명
90 개 이상의 웹툰에 댓글을 단 사람은 1 명
100 개 이상의 웹툰에 댓글을 단 사람은 1 명
110 개 이상의 웹툰에 댓글을 단 사람은 1 명
120 개 이상의 웹툰에 댓글을 단 사람은 0 명
130 개 이상의 웹툰에 댓글을 단 사람은 0 명


### cosine 유사도를 통한 추천 시스템

In [19]:
from scipy.spatial import distance
from sklearn.preprocessing import scale

def find_sim_user(user_id):

    user_wt_matrix = user_df.copy()
    def cosine_distance(a, b):
        return 1 - distance.cosine(a, b)
    vector_1 = user_wt_matrix.copy()
    vector_1 = scale(vector_1)
    scale_df = pd.DataFrame(data = vector_1, index = user_wt_matrix.index, columns = user_wt_matrix.columns)

    empty = np.zeros((1, len(scale_df)))
    scale_df.index = range(len(scale_df))
    userls = list(set(comments['user_id']))
    ls_num_user = pd.DataFrame(userls)
    ls_user_num = pd.DataFrame(data = ls_num_user.index, index = list(ls_num_user[0]))
    ls_user_num = ls_user_num.loc[user_id][0]

    for i in range(len(empty[0])):
        if i != ls_user_num:
            empty[0, i] = cosine_distance(scale_df.loc[ls_user_num], scale_df.loc[i])
    sim_user_num = []
    for _ in range(300):
        a = empty.argmax()
        sim_user_num.append(a)
        empty[0][a] = 0
    sim_user_id = [(ls_num_user.loc[i][0]) for i in sim_user_num]
    return sim_user_id

In [20]:
def is_user_wt(user_id):
    df = pd.DataFrame(user_df.loc[user_id])
    wtls = list(df[df[user_id] != 0].index)
    wt_title = []
    for wtnum in wtls:
        wt_title.append(info_wt[info_wt['wt_num']==wtnum]['title'].values[0])
    return wt_title

In [21]:
def recommend_webtoon(user_id):
    ls = []
    print(user_id,'은', is_user_wt(user_id), '을 즐겨봅니다.')
    for id in find_sim_user(user_id):
        for n in range(len(is_user_wt(id))):
            ls.append(is_user_wt(id)[n])

    df = pd.DataFrame(ls, columns= ['title'])
    df = df.groupby(by='title').size().reset_index(name='count').sort_values('count', ascending=False) 
    for wt in is_user_wt(user_id):
        df = df[df['title'] != wt]
    if len(df) == 0:
        print(user_id,'은 데이터가 적어서 추천할 수 없습니다.')
    return df.reset_index()

In [26]:
df = comments.sample(3)
sample = list(df['user_id'])

In [27]:
for id in sample:
    df = recommend_webtoon(id)
    df = df.drop(['index'],axis=1)
    display(df.head(10))

ni****(o_ob****) 은 ['성공한 덕후', '기기괴괴', '호랑이형님', '조의 영역'] 을 즐겨봅니다.


,title,count
0,연애혁명,7
1,심연의 하늘 시즌5,6
2,그 판타지 세계에서 사는 법,4
3,덴마,3
4,어글리후드,2
5,외모지상주의,2
6,평범한 8반,1
7,진눈깨비 소년,1
8,전자오락수호대,1
9,여신강림,1


냥꾹****(dlek****) 은 ['심연의 하늘 시즌5', '테러맨'] 을 즐겨봅니다.


,title,count
0,연애혁명,4
1,그 판타지 세계에서 사는 법,2
2,호랑이형님,2
3,노블레스,1
4,성공한 덕후,1
5,어글리후드,1
6,외모지상주의,1
7,환상적인 소년,1


흰쥐****(tomm****) 은 ['환상적인 소년'] 을 즐겨봅니다.
흰쥐****(tomm****) 은 데이터가 적어서 추천할 수 없습니다.


,title,count


In [28]:
user_wt_matrix1
empty_matrix = np.zeros((10, len(user_wt_matrix1))) # 샘플 10명
users_index = user_wt_matrix1.index
user_wt_matrix1.index = range(len(user_wt_matrix1))


In [29]:
def cosine_distance(a, b):
        return 1 - distance.cosine(a, b)
for index_1, i in user_wt_matrix1[:10].iterrows():
    for index_2, j in user_wt_matrix1[index_1+1:].iterrows():
        empty_matrix[index_1, index_2] = cosine_distance(i, j)

In [30]:
empty_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
score_list = []
for i in range(10):
    ID_index = []
    while len(ID_index) < 11:
        if empty_matrix[i].argmax() >= 1:
            empty_matrix[i, empty_matrix[i].argmax()] = 0
        else:
            ID_index.append(empty_matrix[i].argmax())
            empty_matrix[i, empty_matrix[i].argmax()] = 0
    data = user_wt_matrix1.loc[i]
    predict = user_wt_matrix1.loc[ID_index].mean()
    score = data[data > 0] - predict[data > 0]
    score_list.append(np.absolute(score).sum()/len(score))
print(np.array(score_list).mean())

0.8800000000000001


In [32]:
score_list

[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8, 1.0, 1.0, 1.0]

In [33]:
empty_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])